# Midterm Project: Classifying German Traffic Signs

I develop a convolutional neural network (CNN) to classify the German Traffic Sign Dataset. The practical implications of this kind of work can be significant, especially in developing self-driving cars or robots that may be able to assess and interpret their surroundings.

I learn from an approach taken in this [this article](https://medium.com/analytics-vidhya/german-traffic-sign-recognition-benchmark-5477ca13daa0) but impose a number of modifications, taking advantage of tools I learn [here](https://towardsdatascience.com/keras-data-generators-and-how-to-use-them-b69129ed779c) and [here](https://towardsdatascience.com/building-a-convolutional-neural-network-cnn-in-keras-329fbbadc5f5):

1. I code up the data intake and modification differently.
2. I remove the cross-validation step and simplify the process, with the goal of hitting the optimal 95% with fewer steps.
3. I use different image data generator.
4. I use a modified CNN model.

Below, I import the libraries for the exercise.

In [1]:
import pandas as pd
import numpy as np
import random
import math
import cv2
import os
import shutil
from google.colab import files
import pathlib
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

## Part 1: Data Intake and Transformation

First, we intake the data from kaggle and import it into Colab after uploading a relevant API key.

In [2]:
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle

!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign
!unzip -q /content/gtsrb-german-traffic-sign.zip -d dataset


Saving kaggle.json to kaggle.json
kaggle.json
 97% 593M/612M [00:05<00:00, 115MB/s]
100% 612M/612M [00:05<00:00, 109MB/s]


Next, we list out the paths to all of the relevant files and perform different transformations on them so we can run our CNN model on them.

For the y-variable cases (the labels), we convert them from a list of numbers (referring to the class) to ndarrays in categorical format. For example, we would have a value of 2 become $[0, 0, 1, 0 \dots, 0].$

In [4]:
# variables defining width, height, channels, epochs, etc
width = 32
height = 32
channels = 3
epochs = 40
num_classes = 43

train_csv = pd.read_csv("/content/dataset/Train.csv")
y_train = keras.utils.to_categorical(train_csv["ClassId"], num_classes)
train_paths = "/content/dataset/"+ train_csv["Path"]

test_csv = pd.read_csv("/content/dataset/Test.csv")
y_test = keras.utils.to_categorical(test_csv["ClassId"], num_classes)
test_paths = "/content/dataset/"+ test_csv["Path"]

For the x-variable cases (the pictures), we convert them from .png files to ndarrays with width and height of 32.

In [5]:
test_images = []
train_images = []

# convert and resize .png files
for i in range(len(test_paths)):
  image = cv2.imread(test_paths[i])
  image = cv2.resize(image, (height,width))
  image = image / 255
  test_images.append(image)

x_test = np.array(test_images)

for i in range(len(train_paths)):
  image = cv2.imread(train_paths[i])
  image = cv2.resize(image, (height,width))
  image = image / 255
  train_images.append(image)

x_train = np.array(train_images)

# convert to float32
x_test = x_test.astype('float32')
x_train = x_train.astype('float32')

## Part 2: Data Generation and Model

We use a simple image data generator using the tool from the keras package and apply some simple data transforms.

In [6]:
datagen = ImageDataGenerator(width_shift_range = 0.1,
                             height_shift_range = 0.1,
                             zoom_range = 0.1,
                             shear_range = 0.1,
                             rotation_range = 10)

datagen.fit(x_train)

Next, we develop simple batches (using batch size of 20).

In [7]:
batches = datagen.flow(x_train, y_train, batch_size = 20)
x_batch, y_batch = next(batches)

Then, we produce a model. This is similar to the default model given for keras but we add an additional set of convolutions and pooling for additional simplification.

In [8]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
    input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

Finally, we initiate the RMSprop optimizer, train the model, and then run the model on the training dataset and the testing dataset. We aim for at least 95% validation accuracy.

In [11]:
# optimizer and model training
opt = keras.optimizers.legacy.RMSprop(learning_rate=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# running the model
history = model.fit_generator(datagen.flow(x_train, y_train, batch_size = 20),
                              steps_per_epoch = 1950,
                              epochs = 20,
                              validation_data =(x_test, y_test),
                              shuffle = 1)

Epoch 1/20
1950/1950 [==============================] - 40s 20ms/step - loss: 3.2353 - accuracy: 0.1187 - val_loss: 2.5784 - val_accuracy: 0.2737
Epoch 2/20
1950/1950 [==============================] - 37s 19ms/step - loss: 2.2297 - accuracy: 0.3432 - val_loss: 1.6914 - val_accuracy: 0.4499
Epoch 3/20
1950/1950 [==============================] - 36s 18ms/step - loss: 1.6768 - accuracy: 0.4585 - val_loss: 1.3075 - val_accuracy: 0.5872
Epoch 4/20
1950/1950 [==============================] - 37s 19ms/step - loss: 1.3297 - accuracy: 0.5620 - val_loss: 0.8799 - val_accuracy: 0.7034
Epoch 5/20
1950/1950 [==============================] - 36s 18ms/step - loss: 1.0355 - accuracy: 0.6591 - val_loss: 0.6772 - val_accuracy: 0.7792
Epoch 6/20
1950/1950 [==============================] - 35s 18ms/step - loss: 0.8064 - accuracy: 0.7357 - val_loss: 0.4261 - val_accuracy: 0.8718
Epoch 7/20
1950/1950 [==============================] - 37s 19ms/step - loss: 0.6380 - accuracy: 0.7940 - val_loss: 0.3486 -

## Part 3: Results

Please see the full breakdown of results in the PDF document including my slides. I give a fuller analysis there and show how I exceed the 95% validation score benchmark.